In [1]:
from LoadData import LoadDataAndProcessing, LoadTestDataAndProcessing
from RumourDataSet import RumourDataset
from torch.utils.data import DataLoader

In [2]:
train_file = "data/train_data_all.json"
train_label_file = "data/train.label.txt"
dev_file = "data/dev_data_all.json"
dev_label_file =  "data/dev.label.txt"

load_twitter_train_data = LoadDataAndProcessing(train_file, train_label_file)
load_twitter_dev_data = LoadDataAndProcessing(dev_file, dev_label_file)

In [5]:
train_input = load_twitter_train_data.prepareDataset()
dev_input = load_twitter_dev_data.prepareDataset()

In [3]:
tweets_file_path = "../../project-data/tweet-objects/tweet-objects/"
tweet_data = "data/test.data.txt"
load_twitter_test_data= LoadTestDataAndProcessing(tweets_file_path, tweet_data)

In [4]:
test_input = load_twitter_test_data.prepareDataset()

In [7]:
import pickle

with open("data/saved/train_input.pickle", "wb") as file_:
    pickle.dump(train_input, file_, -1)

with open("data/saved/dev_input.pickle", "wb") as file_:
    pickle.dump(dev_input, file_, -1)
    
with open("data/saved/test_input.pickle", "wb") as file_:
    pickle.dump(test_input, file_, -1)